# Lab Book for Trajectory Recommendation Experiments

## Multi-label effects on SSVM and RankSVM

Some thoughts:
- Multi-label hurts SSVM as we have multiple labels given an example (a feature vector).
- On the other hand, as the label of example (a POI and query specific feature vector) for RankSVM is computed by counting the number of occurrence of the POI in trajectories that conform to the query, it seems there's no multi-label problem in RankSVM training.

**Experiment**
- Experiment on generated dataset (single label for an example/query)
- Both POI and transition features are scaled linearly to $(-1,1)$ using MinMaxScaler
- $C$ is tuned using Monte-Carlo CV at the beginning and keeped the same for all leave-one-out CV
- $C$ for SSVM, SSVM without transition features and the $C$ for RankSVM are tunned seperatly
- Prediction inference in SSVM is the list Viterbi algorithm
- This experiment is performed in notebook ```generated_data.ipynb```

**Conclusion**
- When features are computed from the original dataset (with duration related features), NOT the generated dataset, SSVM performs better than RankSVM, sample results:
```
RankSVM:           F1 (0.597, 0.011), pairsF1 (0.248, 0.012), Tau (0.507, 0.011)
SSVM:              F1 (0.883, 0.014), pairsF1 (0.667, 0.027), Tau (0.843, 0.017)
SSVM-NoTransition: F1 (0.827, 0.015), pairsF1 (0.510, 0.021), Tau (0.775, 0.015)
```
- When features are recomputed from the generated dataset (so NO duration related features), RankSVM performs better than SSVM, sample results:
```
RankSVM:           F1 (0.903, 0.012), pairsF1 (0.612, 0.023), Tau (0.858, 0.013)
SSVM:              F1 (0.865, 0.016), pairsF1 (0.607, 0.026), Tau (0.822, 0.017)
SSVM-NoTransition: F1 (0.869, 0.017), pairsF1 (0.589, 0.026), Tau (0.819, 0.018) 
```

## Transition feature scaling

**Experiment**
- Scale transition features, i.e. the log probabilities factorised according to a number of features, to range $(-1,1)$ linearly.
- The Osaka dataset is used and the performance is evaluated using leave-one-out cross validation.

**Conclusion**

## SSVM with RankSVM weights

**Experiment** 
- This experiment plug the weights of a trained RankSVM, i.e. a vector $w$ and $|w|$ is the number of POI (and query) features, in SSVM and do prediction, this means we ignore transition features and tie weights among different POIs.
- A dedicated notebook ```ssvm_ranksvm_weights.ipynb``` is created for this experiment.

**Conclusion** 
- We found that SSVM with RankSVM weights performance very similar to RankSVM, in fact, when using the list Viterbi algorithm, the predicted trajectory given any query (in leave-one-out cross validation) contains the exactly same set of POIs as the predition by RankSVM, except the visiting order of these POIs is different.
- Features are scaled using **MinMaxScaler** (scikit-learn), and scaled to range $(-1,1)$ linearly, which is the scaling method used in RankSVM (the libsvm implementation).
- Sample results:
```
As POI features for both RankSVM and SSVM are the same, so we can use the inference procedure of SSVM with RankSVM weights. (All transition features are set to zero)
The weights of RankSVM are computed from the leave-one-out, i.e., a vector of weights for each query in training set.
The dataset used here is Glasgow.
The performance of RankSVM:
F1 (0.767, 0.026), pairsF1 (0.553, 0.047), Tau (0.739, 0.030)
The performance of SSVM using the weights of the above RankSVM:
SSVM-Viterbi: F1 (0.713, 0.032), pairsF1 (0.575, 0.046), Tau (0.740, 0.031)
SSVM-listViterbi: F1 (0.765, 0.026), pairsF1 (0.550, 0.046), Tau (0.739, 0.030)
```
- When the **MaxAbsScaler** was used, SSVM performed much worse.

## SSVM vs RankSVM on generated dataset (Check if multi-label hurts)

**Experiment**
- POI features and transition features are computed on Glasgow dataset, trajectories (i.e., labels) are from generated dataset (#trajectories=125).
- POI and transition features are computed from generated data (NO duration related features)
- Performance are evaluated using leave-one-out cross validation.
- Regularisation parameter $C$ are tuned using Monte-Carlo cross validation at the beginning, then the same $C$ was used for all leave-one-out cross validations.
- Prediction inference in SSVM is the list Viterbi algorithm
- A dedicated notebook ```generated_data.ipynb``` is created for this experiment.

**Conclusion**
- Neither SSVM nor RankSVM can achieve nearly perfect performance, which mean multi-label problem hurt both models.
- RankSVM performs better than SSVM.
- Sample results:
```
SSVM:    F1 (0.699, 0.015), pairsF1 (0.416, 0.021), Tau (0.643, 0.016)
RankSVM: F1 (0.881, 0.011), pairsF1 (0.572, 0.019), Tau (0.829, 0.012)
```